In [1]:
import pandas as pd
import json
import ast
from textblob import TextBlob
import re



ETL

Se realiza la carga y transformaciones necesarias de los archivos 
- 'australian_user_reviews.json'
- 'australian_users_items.json'
- 'output_steam_games.json'

Despues de este del proceso de ETL, se obtuvieron los sgtes csv 'steam.csv', 'df_items.csv' y 'reviuws.csv'

DATA  'australian_user_reviews.json'

In [ ]:
# Se abre el archivo en formato json "steam_games" y se lee el archivo linea por linea y aplicamos la funcion json.loads
# a cada una de las lineas
data = []
with open('output_steam_games.json') as steam:
    for linea in steam:
        dato = json.loads(linea)
        data.append(dato) 

In [ ]:
steam_game = pd.DataFrame(data)
print(steam_game.shape)
print(steam_game.head())


In [ ]:
# # Se realiza un checkeo general del dataframe y asi tener mejor comprension de los campos que contiene
steam_game.info()


In [ ]:
# Despues de mirar cada uno de los campos se eliminan los inecesarios y se consideran campos especificos,
# los cuales son requeridos para el desarrollo de las funciones

steam = steam_game[['id', 'genres','app_name','release_date']].copy()

In [ ]:
steam.shape

In [ ]:
# Observamos el nuevo Dataframe 'steam'
steam.info()

In [ ]:
# El dataframe tiene aproximadamente mas del 70% de nulos en comparacion al total de los registros
steam.isnull().sum()

In [ ]:
# Se eliminan los nulos ya que no son representativas para el desarrollo del proyecto 
# Generando un dataframe con 28.832 registro validos
steam.dropna(inplace=True)

In [ ]:
# Se trabaja cada uno de los campos del dataframe
# Se realiza el cambio de tipo de dato a int en el campo 
steam['id'] = steam['id'].astype(int)


In [ ]:
# En el campo 'genres', se puede observar que esta compuesto por listas de generos x juegos
steam['genres'].sample(3)

In [ ]:
# Observamos como los valores unicos
steam['genres'].explode().unique()

In [ ]:
# En el campo 'release_date', por lo que son fechas los registros, se tiene un tipo de dato no valido
steam['release_date'].head(3)

In [ ]:
steam['release_date'].unique()

In [ ]:
# Esta funcion tomara esos registros que solo tienen el año y los convertira en formato Y-M-D
from datetime import datetime

def procesar_cadena(cadena):
    
    if isinstance(cadena, str):
        try:
            # Intenta encontrar un año en la cadena utilizando una expresión regular
            import re
            year_match = re.search(r'\b\d{4}\b', cadena)
            
            if year_match:
                # Si se encuentra un año, conviértelo al formato '%Y-%m-%d'
                year = int(year_match.group())
                return datetime(year, 1, 1).strftime('%Y-%m-%d')
            else:
                # Si no se encuentra un año, devuelve None
                return None
        except ValueError:
            # Si hay un error en la conversión, devuelve None
            return None
    else:
        # Si no es una cadena, devuelve None
        return None



In [ ]:
steam['release_date'] = steam['release_date'].apply(procesar_cadena)


In [ ]:
steam['release_date'].head(3)

In [ ]:
steam.dropna(subset=['release_date'], inplace=True)


In [ ]:
# Se procede a realizar el cambio
steam['release_date'] = pd.to_datetime(steam['release_date'], errors='coerce')


In [ ]:
steam['release_date']

In [ ]:
# Se crea una nueva columna 'release_year', donde se extrae el año de especifico de lanzamiento
steam.loc[:, 'year'] = steam['release_date'].dt.year
steam['year'] = steam['year'].astype('int64')


In [ ]:
steam.info()

In [ ]:
steam.to_csv('steam.csv')

DATA 'australian_users_items.json'

In [ ]:
data1 = []
with open('australian_users_items.json',encoding='utf-8') as item:
    for linea in item.readlines():
        data1.append(ast.literal_eval(linea))

In [ ]:
#Convertimos a Dataframe
item = pd.DataFrame(data1)

In [ ]:
item.head()

In [ ]:
# En el campo 'item' se encuentran listas de diccionarios, se utiliza la funcion 'json_normsliza' para desanidar
df_items = pd.json_normalize(data1, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )


In [ ]:
# Observamos el dataframe
df_items.head()

In [ ]:
# Se eliminan los campos innecesarios 'playtime_2weeks', 'user_url', 'item_name','items_count','steam_id'
df_items = df_items.drop(['user_url', 'playtime_2weeks'], axis = 1)

In [7]:
df_items = df_items.drop(['item_name','items_count','steam_id'], axis = 1)

In [24]:
df_items = df_items.drop(columns=['Unnamed: 0'])


In [21]:
df_items = pd.read_csv('df_items.csv')

In [15]:
df_items = df_items.drop_duplicates()


In [55]:
# Se toma la decision de trabajar con el 96% de los datos, para disminuir su tamaño de memoria
sample_size = int(0.65 * len(df_items)) 
df_items = df_items.sample(n=sample_size, random_state=42) 

In [56]:
df_items.shape

(3210544, 3)

In [57]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3210544 entries, 4559349 to 1794319
Data columns (total 3 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   playtime_forever  int64 
 2   user_id           object
dtypes: int64(2), object(1)
memory usage: 98.0+ MB


DATA 'australian_user_reviews.json'

In [ ]:
# Se abre el archivo 'australian_user_reviews'
data2 = []
with open('australian_user_reviews.json',encoding='utf-8') as reviuw:
    for linea in reviuw.readlines():
        data2.append(ast.literal_eval(linea))

In [ ]:
#Convertimos a Dataframe
reviuw = pd.DataFrame(data2)

In [ ]:
# Se checkea como esta compuesto, y se observa que campo 'reviews' esta anidado
reviuw.head()

In [ ]:
# Por medio de este For, se puede acceder a los campos dentro del campo reviews y obtener un nuevo Dataframe
data_des = []
for i, e in reviuw.iterrows():
    user_id = e['user_id']
    user_url = e['user_url']
    for reseñas in e['reviews']:
        reseñas['user_id'] = user_id
        reseñas['user_url'] = user_url
        data_des.append(reseñas)

df_reviuw = pd.DataFrame(data_des)

In [ ]:
df_reviuw.head()

In [ ]:
df_reviuw.info()

In [ ]:
# Se crea la funcion sentimiento para  aplicar análisis de sentimiento con NLP 
# debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo
def sentiment(data):
    npl = TextBlob(data)
    if npl.sentiment.polarity < 0:
        return 0
    elif npl.sentiment.polarity == 0:
        return 1
    else:
        return 2

In [ ]:
df_reviuw['sentiment_analisy']=df_reviuw['review'].apply(sentiment)

In [ ]:
# Se elimina el campo 'review'
df_reviuw = df_reviuw.drop('review', axis=1)

In [ ]:
df_reviuw.head()

In [ ]:
#Se crea una funcion para cambiar el formato del campo posted a formato "YYYY-MM-DD"
def convertir_fecha(fecha):
    match = re.search(r'(\w+\s\d{1,2},\s\d{4})', fecha)
    if match:
        fecha_str = match.group(1)
        try:
            fecha_dt = pd.to_datetime(fecha_str)
            return fecha_dt.strftime('%Y-%m-%d')
        except:
            return 'Fecha inválida'
    else:
        return 'Formato inválido'

In [ ]:
#Se crea un nuevo campo 'date'
df_reviuw['date'] = df_reviuw['posted'].apply(convertir_fecha)

In [ ]:
#Se Elimina 'posted'
df_reviuw = df_reviuw.drop('posted', axis=1)


In [ ]:
df_reviuw.head()

In [ ]:
df_reviuw.shape

In [ ]:
df_reviuw.to_csv('reviuws.csv')